In [3]:
!pip install scikit-learn==1.0.2 pandas==1.3.5 numpy==1.21.5 matplotlib==3.5.1 seaborn==0.11.2 joblib==1.1.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 70.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (pyproject.toml) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

def load_data():
    data_url = "http://lib.stat.cmu.edu/datasets/boston"
    raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
    data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
    target = raw_df.values[1::2, 2]
    feature_names = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT']
    df = pd.DataFrame(data, columns=feature_names)
    df['MEDV'] = target
    return df

df = load_data()
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [5]:
X = df.drop('MEDV', axis=1)
y = df['MEDV']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [6]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

models = {
    "Linear Regression": LinearRegression(),
    "Decision Tree": DecisionTreeRegressor(random_state=42),
    "Random Forest": RandomForestRegressor(random_state=42)
}

results = []

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    results.append({"Model": name, "MSE": mse, "R2 Score": r2})

results_df = pd.DataFrame(results)
results_df

,Model,MSE,R2 Score
0,Linear Regression,24.291119,0.668759
1,Decision Tree,10.416078,0.857963
2,Random Forest,7.912745,0.892100


In [7]:
from sklearn.model_selection import GridSearchCV

param_grids = {
    "Linear Regression": {
        "fit_intercept": [True, False],
        "copy_X": [True, False],
        "n_jobs": [None, -1]
    },
    "Decision Tree": {
        "max_depth": [None, 5, 10, 20],
        "min_samples_split": [2, 5, 10],
        "min_samples_leaf": [1, 2, 4]
    },
    "Random Forest": {
        "n_estimators": [50, 100, 200],
        "max_depth": [None, 10, 20],
        "min_samples_split": [2, 5, 10]
    }
}

tuned_results = []

for name, model in models.items():
    print(f"\nTuning {name}...")
    grid_search = GridSearchCV(
        model,
        param_grids[name],
        cv=5,
        scoring="neg_mean_squared_error",
        n_jobs=-1,
        verbose=1
    )
    grid_search.fit(X_train, y_train)
    best_model = grid_search.best_estimator_
    y_pred = best_model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    tuned_results.append({
        "Model": name,
        "Best Parameters": grid_search.best_params_,
        "MSE": mse,
        "R2 Score": r2
    })

tuned_results_df = pd.DataFrame(tuned_results)
tuned_results_df


Tuning Linear Regression...
Fitting 5 folds for each of 8 candidates, totalling 40 fits

Tuning Decision Tree...
Fitting 5 folds for each of 36 candidates, totalling 180 fits

Tuning Random Forest...
Fitting 5 folds for each of 27 candidates, totalling 135 fits


,Model,Best Parameters,MSE,R2 Score
0,Linear Regression,"{'copy_X': True, 'fit_intercept': True, 'n_job...",24.291119,0.668759
1,Decision Tree,"{'max_depth': 5, 'min_samples_leaf': 2, 'min_s...",9.343752,0.872586
2,Random Forest,"{'max_depth': None, 'min_samples_split': 2, 'n...",7.912745,0.892100


In [8]:
import joblib

for name, model in models.items():
    joblib.dump(model, f"{name.replace(' ', '_').lower()}_model.pkl")